In [2]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
])
train_data = datasets.CIFAR10(
    root="data",
    train=True,
    download=True,
    transform=transform
)
test_data = datasets.CIFAR10(
    root="data",
    train=False,
    download=True,
    transform=transform
)

batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3,16,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(16,32,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.fc = nn.Linear(32*8*8, 10)
    def forward(self, x):
        x = self.net(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

model = Model().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
epochs = 10

for epoch in range(epochs):
    print(f'Epoch: {epoch}')
    for batch_idx, (X, Yd) in enumerate(train_loader):
        optimizer.zero_grad()
        X = X.to(device)
        Yd = Yd.to(device)
        Y_logit = model(X)
        loss = F.cross_entropy(Y_logit, Yd)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            Y = torch.argmax(Y_logit, dim=1)
            correct = (Yd == Y).sum().item() 
            total = Yd.size(0)
            acc = correct / total
            print(f"Batch idx: {batch_idx}, Train-Loss: {loss.item():.3f}, Train-Acc: {acc:.2f}")
        
    with torch.no_grad():
        loss_all = 0
        correct, total = 0, 0
        for X, Yd in test_loader:
            X = X.to(device)
            Yd = Yd.to(device)
            Y_logit = model(X)
            loss = F.cross_entropy(Y_logit, Yd)
            loss_all += loss.item()
            Y = torch.argmax(Y_logit, dim=1)
            correct += (Yd == Y).sum().item() 
            total += Yd.size(0)
        loss_all /= len(test_loader)
        acc = correct / total
        print(f"Test-Loss: {loss_all:.3f}, Test-Acc: {acc:.2f}")


Files already downloaded and verified
Files already downloaded and verified
cuda
Epoch: 0
Batch idx: 0, Train-Loss: 2.306, Train-Acc: 0.08
Batch idx: 100, Train-Loss: 1.621, Train-Acc: 0.31
Batch idx: 200, Train-Loss: 1.300, Train-Acc: 0.59
Batch idx: 300, Train-Loss: 1.555, Train-Acc: 0.42
Batch idx: 400, Train-Loss: 1.193, Train-Acc: 0.53
Batch idx: 500, Train-Loss: 1.392, Train-Acc: 0.58
Batch idx: 600, Train-Loss: 1.359, Train-Acc: 0.52
Batch idx: 700, Train-Loss: 0.980, Train-Acc: 0.72
Test-Loss: 1.258, Test-Acc: 0.55
Epoch: 1
Batch idx: 0, Train-Loss: 1.270, Train-Acc: 0.53
Batch idx: 100, Train-Loss: 0.932, Train-Acc: 0.69
Batch idx: 200, Train-Loss: 0.856, Train-Acc: 0.70
Batch idx: 300, Train-Loss: 0.953, Train-Acc: 0.70
Batch idx: 400, Train-Loss: 1.175, Train-Acc: 0.62
Batch idx: 500, Train-Loss: 1.109, Train-Acc: 0.62
Batch idx: 600, Train-Loss: 0.938, Train-Acc: 0.70
Batch idx: 700, Train-Loss: 1.016, Train-Acc: 0.66
Test-Loss: 1.102, Test-Acc: 0.61
Epoch: 2
Batch idx: 0, 